In [5]:
import pandas as pd
import numpy as np
import gc

import sklearn
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import imblearn
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier

In [7]:
data_type = "train"
# data_type2 = "test"
# month = "07"
# category = "잔액정보"

# local
root_path = '../data/open'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

drive_folder = f'{root_path}/{data_type1}/5.잔액정보/'
# drive_folder2 = f'{root_path}/{data_type2}/5.잔액정보/'

## Modeling(1) - Feature importance

In [9]:
# 1. 데이터 불러기
train_df = pd.read_parquet(f'{drive_folder}train_잔액정보_통합_전처리1.parquet')

In [4]:
# 2. 피처/타겟 분리 (ID, Segment 제외)
feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]
X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()

In [5]:
# 3. 타겟 인코딩 (문자 → 숫자)
y = y.map({'A':0, 'B':1, 'C':2, 'D':3, 'E':4})

In [6]:
# 4. 메모리 정리
del train_df
gc.collect()

0

In [7]:
# 5. 클래스 weight 계산 --> 쓸모 없을지도 수정안해서..
classes = np.unique(y)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
class_weights = dict(zip(classes, weights))

In [8]:
# 6. 각 샘플에 대해 weight 매핑 --> 쓸모 없을지도 수정안서서
w_train = y.map(class_weights)

In [9]:
# 7. XGBoost 모델 학습 (전체 피처 기준 중요도 추출용)
temp_model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=5,
    eval_metric='mlogloss',
    n_estimators=700,
    tree_method='hist',
    device='cuda',
    random_state=42
)

temp_model.fit(X, y, sample_weight=w_train, verbose=False)

C:\Users\win11\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:07:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\win11\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:07:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=700, n_jobs=None, num_class=5, ...)

In [10]:
# 변수 중요도 정리
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': temp_model.feature_importances_
}).sort_values(by='importance', ascending=False)

In [11]:
# top N개 변수 선택 (여기선 50개)
topN = 50
top_features = importance_df.head(topN)['feature'].tolist()

In [12]:
print(f"🎯 상위 {topN}개 중요 변수:")
print(top_features)

🎯 상위 50개 중요 변수:
['잔액_할부_무이자_B0M', '평잔_6M', '월중평잔_일시불_B0M', '평잔_일시불_해외_6M', '평잔_CA_6M', '평잔_CA_3M', '평잔_3M', '평잔_할부_6M', '평잔_일시불_해외_3M', '평잔_RV일시불_해외_3M', '잔액_리볼빙CA이월_B0M', '잔액_현금서비스_B0M', '평잔_카드론_3M', '평잔_RV일시불_해외_6M', '월중평잔', 'RV_최대잔액_R3M', '평잔_할부_3M', '잔액_리볼빙일시불이월_B0M', '월중평잔_할부_B0M', '잔액_현금서비스_B2M', '잔액_카드론_B2M', '잔액_할부_B0M', '월중평잔_일시불', '월중평잔_카드론', '잔액_할부_유이자_B0M', 'RV_평균잔액_R3M', '최종연체회차', '평잔_카드론_6M', '월중평잔_CA', '평잔_RV일시불_3M', '잔액_할부_B2M', '평잔_일시불_6M', 'RV_평균잔액_R6M', '평잔_RV일시불_6M', '월중평잔_RV일시불', '잔액_일시불_B0M', 'RV_최대잔액_R6M', '잔액_카드론_B0M', '잔액_카드론_B5M', '잔액_일시불_B2M', 'RV_평균잔액_R12M', '잔액_일시불_B1M', '잔액_현금서비스_B1M', '평잔_일시불_3M', '잔액_할부_B1M', 'RV_최대잔액_R12M', '월중평잔_CA_B0M', '월중평잔_할부', '평잔_할부_해외_6M', '잔액_카드론_B3M']


In [13]:
# CSV 저장 (경로는 로컬 기준으로 수정)
top_df = pd.DataFrame({'feature': top_features})
top_df.to_csv(f'{drive_folder}top_features_XGB_balanced.csv', index=False, encoding='utf-8-sig')

## Modeling(2) - Final model train

In [15]:
# ✅ top 피처 불러오기
top_feats_df = pd.read_csv(f'{drive_folder}top_features_XGB_balanced.csv')  # 저장한 피처 리스트
top_features = top_feats_df['feature'].tolist()

In [16]:
# ✅ top 피처만 선택
X_top = X[top_features]  # 기존 X에서 선택
y_top = y 

In [20]:
# 3. train/validation 나누기 (성능 확인용)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_top, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
# # ✅ 오버샘플링 (A=0, B=1, C=2만 증강)
# smote = SMOTE(sampling_strategy={0: 5000, 1: 5000, 2: 150000}, random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X_top, y)

In [ ]:
# # ✅ 클래스 가중치 계산
# classes = np.unique(y_resampled)
# weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_resampled)
# class_weights = dict(zip(classes, weights))
# sample_weights = pd.Series(y_resampled).map(class_weights)

In [ ]:
# # ✅ 클래스별 weight 출력
# inverse_label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
# print("📊 클래스별 가중치:")
# for cls in sorted(class_weights):
#     print(f"클래스 {inverse_label_map[cls]} ({cls}): weight = {class_weights[cls]:.2f}")

In [22]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=5,
    eval_metric='mlogloss',
    n_estimators=700, 
    tree_method='hist',
    device='cuda',
    random_state=42
    )

In [24]:
# # 모델 학습 (검증 없이 전체 데이터 사용)
# xgb_model.fit(
#     X_resampled, y_resampled,
#     sample_weight=sample_weights,
#     verbose=False
# )

In [26]:
# 5. 모델 학습
xgb_model.fit(X_train, y_train, verbose=False)

C:\Users\win11\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:26:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\win11\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:26:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=700, n_jobs=None, num_class=5, ...)

In [35]:
# 6. 검증 성능 확인
y_pred = xgb_model.predict(X_valid)
acc = accuracy_score(y_valid, y_pred)

In [36]:
print(f"🎯 Validation Accuracy: {acc:.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_valid, y_pred))

🎯 Validation Accuracy: 0.8469

📋 Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.22      0.35       194
           1       1.00      0.07      0.13        29
           2       0.64      0.41      0.50     25518
           3       0.55      0.37      0.44     69848
           4       0.89      0.96      0.92    384411

    accuracy                           0.85    480000
   macro avg       0.77      0.41      0.47    480000
weighted avg       0.83      0.85      0.83    480000



In [37]:
xgb_model.save_model(f'{drive_folder}softvoting_xgb_xgb변수_3_3_25.json')

## 예측 / Soft Voting

In [41]:
test_df = pd.read_parquet(f'{drive_folder}test_잔액정보_통합_전처리1.parquet')

In [43]:
top_feats_df = pd.read_csv(f'{drive_folder}top_features_XGB_balanced.csv')  # ← 저장했던 top50 피처 목록
top_features = top_feats_df['feature'].tolist()

In [45]:
# 1. 모델 불러오기
xgb_model_loaded = xgb.XGBClassifier()
xgb_model_loaded.load_model(f'{drive_folder}softvoting_xgb_xgb변수_3_3_25.json')  # 로컬 경로

In [47]:
# 2. test 데이터 준비
X_test = test_df[top_features]  # top30 or top300 리스트 중 택1

In [49]:
# 3. 예측 확률
proba_xgb = xgb_model_loaded.predict_proba(X_test)  # iteration_range 제거

In [50]:
# 4. 소프트보팅 구성
ensemble_proba = proba_xgb  # 단일 모델 사용

In [51]:
# 5. 최종 예측
ensemble_preds = np.argmax(ensemble_proba, axis=1)
inverse_label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
ensemble_preds_label = pd.Series(ensemble_preds).map(inverse_label_map)

In [52]:
# 6. 예측값 test_df에 추가
test_df["pred_label"] = ensemble_preds_label

In [53]:
# 7. ID 중복 없다면 그대로 저장
submission = test_df[["ID", "pred_label"]].rename(columns={"pred_label": "Segment"})

In [54]:
# 8. 저장
submission.to_csv(
    f'{drive_folder}submission_xgb.csv',  # 로컬 저장
    index=False,
    encoding="utf-8-sig"
)

print("✅ 결과 저장 완료!")

✅ 결과 저장 완료!


In [11]:
submission = pd.read_csv(f'{drive_folder}submission_xgb.csv')
submission

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,E
3,TEST_00003,E
4,TEST_00004,E
...,...,...
599995,TEST_99995,E
599996,TEST_99996,E
599997,TEST_99997,E
599998,TEST_99998,D


In [56]:
submission['Segment'].value_counts()

Segment
E    521087
D     58666
C     20207
A        37
B         3
Name: count, dtype: int64

In [57]:
submission = test_df.groupby("ID")["pred_label"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

submission.columns = ["ID", "Segment"]

In [58]:
submission.to_csv(f'{drive_folder}submission_xgb_clean.csv', index=False, encoding="utf-8-sig")

print("✅ 중복 제거 후 제출 파일 저장 완료!")

✅ 중복 제거 후 제출 파일 저장 완료!


In [13]:
# 제출용 파일 확인
submission = pd.read_csv(f'{drive_folder}submission_xgb_clean.csv')

print("🧾 제출 파일 ID 개수:", submission["ID"].nunique())
print("📦 제출 파일 총 행 수:", len(submission))

🧾 제출 파일 ID 개수: 100000
📦 제출 파일 총 행 수: 100000
